# Contrastive Learning From Scratch

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [1]:
import os

# Set Project home
#PROJECT_HOME = os.path.join('/',
#                            'workspace',
#                            'gatech',
#                            'cs7643-deep-learning',
#                            'contrastive-learning-in-distilled-models')
PROJECT_HOME = os.path.join('/',
                            'Users',
                            'ng-ka',
                            'OMSCS',
                            'DL',
                            'DLProject',
                            'contrastive-learning-in-distilled-models')
%cd {PROJECT_HOME}

# Load project code
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')

#import distilface
import src.distilface as distilface

C:\Users\ng-ka\OMSCS\DL\DLProject\contrastive-learning-in-distilled-models


## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [18]:
#size = 1000000
size = 100
train_size = int(0.75*size)
train_name = "data/training/wiki"+str(train_size)+"_train_for_simcse.txt"
valid_name = "data/training/wiki"+str(size-train_size)+"_valid_for_simcse.txt"

from itertools import islice
with open("data/training/wiki1m_for_simcse.txt", "r", encoding="utf8") as myfile:
    train_data = list(islice(myfile, train_size))

# always remember, use files in a with statement
with open(train_name, "w",encoding="utf8") as f2:
    for item in train_data:
        f2.write(item)
        
with open("data/training/wiki1m_for_simcse.txt", "r", encoding="utf8") as myfile:
    train_data = list(islice(myfile, train_size,size))
    
# always remember, use files in a with statement
with open(valid_name, "w",encoding="utf8") as f2:
    for item in train_data:
        f2.write(item)

In [2]:
#size = 1000000
size = 100
train_size = int(0.75*size)
train_name = "data/training/wiki"+str(train_size)+"_train_for_simcse.txt"
valid_name = "data/training/wiki"+str(size-train_size)+"_valid_for_simcse.txt"

import numpy as np

from datasets import load_dataset

data_files = {'train': train_name, 'validation': valid_name}
# data_files = {'train': 'data/training/wiki5k.txt'}
#datasets = load_dataset('text', data_files=data_files, cache_dir='./data/')
datasets = load_dataset('text', data_files=data_files)

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default-933581723230cc23
Reusing dataset text (C:\Users\ng-ka\.cache\huggingface\datasets\text\default-933581723230cc23\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 181.79it/s]


In [3]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

print('column_names:', column_names)
print('sent0_cname:', sent0_cname, '| sent1_cname:', sent1_cname)

column_names: ['text']
sent0_cname: text | sent1_cname: text


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "
    
    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=32,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [6]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)
validation_dataset = datasets["validation"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)

Loading cached processed dataset at C:\Users\ng-ka\.cache\huggingface\datasets\text\default-933581723230cc23\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8\cache-3727a2900f89dabb.arrow
Loading cached processed dataset at C:\Users\ng-ka\.cache\huggingface\datasets\text\default-933581723230cc23\0.0.0\4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8\cache-12e35541180b4e8c.arrow


In [7]:
train_dataset.num_rows

75

In [8]:
str(train_dataset['input_ids'][0][0])

'[101, 26866, 1999, 2148, 2660, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [ ]:
str(train_dataset['input_ids'][0][1])

In [ ]:
train_dataset.features.keys()

Sentence 1 and Sentence 2 are the same sentence

## 2. Contrastive Learning Model

In [9]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from src.distilface.modules.pooler import Pooler
from src.distilface.modules.similarity import Similarity


class BertCLModel(BertPreTrainedModel):
    def __init__(self, config, pooler_type='avg_first_last', temp=0.05):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = 0.05

        self.bert = BertModel(config, add_pooling_layer=False)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None):
        if self.training:
            return self.cl_forward(self.bert, input_ids, attention_mask, token_type_ids)
        else:
            return self.sent_emb(self.bert, input_ids, attention_mask, token_type_ids)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        batch_size = input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences
        
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)
        token_type_ids = token_type_ids.view((-1, token_type_ids.size(-1))) # (bs * num_sent, len)

        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )
        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None, token_type_ids=None):
        
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)
        token_type_ids = token_type_ids.view((-1, token_type_ids.size(-1))) # (bs * num_sent, len)
        
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            output_attentions=True,
            output_hidden_states=True,
            return_dict=True,
        )

        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'bert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = BertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

model.eval();


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertCLModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'bert.pooler.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 2.1 Initial BERT embeddings performance

In [10]:
import senteval


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    return results

In [18]:
results = evaluate_model()
#results

C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\senteval\sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent1 = np.array([s.split() for s in sent1])[not_empty_idx]
C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\senteval\sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sent2 = np.array([s.split() for s in sent2])[not_empty_idx]


In [ ]:
results

In [20]:
    print(results["STS12"]["all"]["spearman"].keys())
    print('STS12 mean: ', results["STS12"]["all"]["spearman"])
    print('STS12 wmean: ', results["STS12"]["all"]["spearman"])
    print('STS13 mean: ', results["STS13"]["all"]["spearman"]["mean"])
    print('STS13 wmean: ', results["STS13"]["all"]["spearman"]["wmean"])
    print('STS14 mean: ', results["STS14"]["all"]["spearman"]["mean"])
    print('STS14 wmean: ', results["STS14"]["all"]["spearman"]["wmean"])
    print('STS15 mean: ', results["STS15"]["all"]["spearman"]["mean"])
    print('STS15 wmean: ', results["STS15"]["all"]["spearman"]["wmean"])
    print('STSB: ', results["STSBenchmark"]["spearman"])

dict_keys(['mean', 'wmean'])
STS12 mean:  {'mean': 0.5468686878940625, 'wmean': 0.5443486582913551}
STS12 wmean:  {'mean': 0.5468686878940625, 'wmean': 0.5443486582913551}
STS13 mean:  0.5450458580607989
STS13 wmean:  0.5802891692744219
STS14 mean:  0.5882437430679742
STS14 wmean:  0.588747600067705
STS15 mean:  0.6736581656213602
STS15 wmean:  0.6794191674814121
STSB:  0.7117942971683138


## 3. Trainer

In [11]:
import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=5e-05,
    weight_decay=0.0,
    num_train_epochs=1,
    eval_steps=100,
)

In [12]:
#model.train()

def model_init():
    pretrained_model_name = 'bert-base-uncased'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    config = AutoConfig.from_pretrained(pretrained_model_name)
    model = BertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
    return model

def my_objective(metrics):
    return metrics["eval_loss"]

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# from transformers import EvalPrediction
# def compute_metrics(p: EvalPrediction):
#     preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
#     preds = np.argmax(preds, axis=1)
#     return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics= compute_metrics
)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-u

In [19]:
#pip install "ray[tune]"
#pip install hyperopt
#pip install optuna

In [22]:
#os.environ["TUNE_MAX_PENDING_TRIALS_PG"] = "1"

In [12]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer.hyperparameter_search(
    direction="minimize", 
    backend="optuna",
    #compute_objective=my_objective
#     resources_per_trial={
#         "cpu": 1,
#         "gpu": 1
#     },
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    #search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=ASHAScheduler(metric="objective", mode="max"),
    #n_trials=8 # number of trials
)

[I 2022-03-26 01:59:33,349] A new study created in memory with name: no-name-5f23b8e3-5b6d-4181-b349-f205acf147cd
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "t

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8
[W 2022-03-26 01:59:37,078] Trial 0 failed because of the following error: KeyError('eval_loss')
Traceback (most recent call last):
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\integrations.py", line 158, in _objective
    trainer.objective = trainer.compute_objective(metrics)
  File "C:\Users\ng-ka\AppData\Local\Temp\ipykernel_10524\4209526589.py", line 13, in my_objective
    return metrics["eval_loss"]
KeyError: 'eval_loss'


KeyError: 'eval_loss'

In [14]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer.hyperparameter_search(
    direction="maximize", 
    backend="optuna"#,
    #compute_objective=my_objective
#     resources_per_trial={
#         "cpu": 1,
#         "gpu": 1
#     },
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    #search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=ASHAScheduler(metric="objective", mode="max"),
    #n_trials=8 # number of trials
)

[I 2022-03-26 01:20:20,051] A new study created in memory with name: no-name-a99ab3dc-10de-4ee0-8e6c-03e2e03bf9fd
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "t

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8


evaluate trials
BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.1106, -0.2462, -0.5635,  ..., -0.3272,  0.4589,  0.5335],
         [-0.0938, -0.4328, -0.1038,  ..., -0.4340,  0.3108, -0.0326],
         [-0.3895,  0.1125, -0.9434,  ..., -0.2336, -0.2477, -0.2000],
         ...,
         [ 0.1991,  0.0874, -0.0233,  ..., -0.1527,  0.0203, -0.3245],
         [ 0.3433,  0.1391,  0.0133,  ..., -0.0945, -0.1172, -0.2706],
         [ 0.3886, -0.2084,  0.1146,  ..., -0.1066,  0.0879, -0.0374]],

        [[-0.1106, -0.2462, -0.5635,  ..., -0.3272,  0.4589,  0.5335],
         [-0.0938, -0.4328, -0.1038,  ..., -0.4340,  0.3108, -0.0326],
         [-0.3895,  0.1125, -0.9434,  ..., -0.2336, -0.2477, -0.2000],
         ...,
         [ 0.1991,  0.0874, -0.0233,  ..., -0.1527,  0.0203, -0.3245],
         [ 0.3433,  0.1391,  0.0133,  ..., -0.0945, -0.1172, -0.2706],
         [ 0.3886, -0.2084,  0.1146,  ..., -0.1066,  0.0879, -0.0374]],

        [[-0.0464, -0.1074,  0.0121,  ..., -0.3355,  0.

evaluate trials
BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.2531,  0.0180, -0.2632,  ...,  0.0076,  0.2578,  0.5016],
         [ 0.0616,  0.1561, -0.5806,  ...,  0.0873,  0.6580,  0.0937],
         [-0.3186, -0.2305, -0.3577,  ...,  0.3526, -0.1710, -0.0380],
         ...,
         [-0.4423, -0.6505, -0.2774,  ...,  0.4476,  0.0275, -0.2425],
         [ 0.1962,  0.0136, -0.0334,  ...,  0.3350, -0.1302, -0.1040],
         [ 0.1378, -0.0316, -0.0877,  ...,  0.2801, -0.1191, -0.0711]],

        [[-0.2531,  0.0180, -0.2632,  ...,  0.0076,  0.2578,  0.5016],
         [ 0.0616,  0.1561, -0.5806,  ...,  0.0873,  0.6580,  0.0937],
         [-0.3186, -0.2305, -0.3577,  ...,  0.3526, -0.1710, -0.0380],
         ...,
         [-0.4423, -0.6505, -0.2774,  ...,  0.4476,  0.0275, -0.2425],
         [ 0.1962,  0.0136, -0.0334,  ...,  0.3350, -0.1302, -0.1040],
         [ 0.1378, -0.0316, -0.0877,  ...,  0.2801, -0.1191, -0.0711]],

        [[-0.0951,  0.1491, -0.1526,  ..., -0.4010,  0.

...

evaluate trials
BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 1.1830e-01,  2.6755e-01, -5.4004e-01,  ...,  1.4662e-01,
           9.0053e-02,  3.1347e-01],
         [ 2.2560e-01, -1.6542e-01,  2.5995e-02,  ...,  1.3874e-01,
           3.4007e-01,  8.5501e-01],
         [ 6.1818e-01,  6.6266e-01, -5.4409e-01,  ...,  9.5837e-01,
          -2.8103e-01,  2.0609e-02],
         ...,
         [-4.1178e-01, -2.2249e-01,  2.3812e-01,  ...,  2.8420e-01,
           1.9291e-01,  6.0341e-01],
         [-3.0063e-01,  1.5860e-04,  2.7516e-01,  ...,  2.8905e-01,
           3.9215e-01,  4.9266e-01],
         [-4.0837e-01, -9.3404e-02,  2.9836e-01,  ...,  2.7447e-01,
           3.3021e-01,  5.2722e-01]],

        [[ 1.1830e-01,  2.6755e-01, -5.4004e-01,  ...,  1.4662e-01,
           9.0053e-02,  3.1347e-01],
         [ 2.2560e-01, -1.6542e-01,  2.5995e-02,  ...,  1.3874e-01,
           3.4007e-01,  8.5501e-01],
         [ 6.1818e-01,  6.6266e-01, -5.4409e-01,  ...,  9.5837e-01,
          -2.81

evaluate trials
BaseModelOutputWithPooling(last_hidden_state=tensor([[[-0.1783,  0.0757,  0.3210,  ..., -0.6267,  0.2842,  0.0654],
         [ 0.3826,  0.3287,  0.4941,  ..., -0.4334,  0.3943, -0.2983],
         [ 0.1732,  0.9288,  1.2006,  ..., -0.7994,  0.6691, -1.1489],
         ...,
         [ 0.0438,  0.2019,  0.4674,  ..., -0.1616,  0.3687, -0.2798],
         [ 0.0242,  0.0713,  0.5750,  ..., -0.2297,  0.3579, -0.2498],
         [ 0.5444, -0.1255,  0.3838,  ..., -0.2626,  0.0704, -0.6563]],

        [[-0.1783,  0.0757,  0.3210,  ..., -0.6267,  0.2842,  0.0654],
         [ 0.3826,  0.3287,  0.4941,  ..., -0.4334,  0.3943, -0.2983],
         [ 0.1732,  0.9288,  1.2006,  ..., -0.7994,  0.6691, -1.1489],
         ...,
         [ 0.0438,  0.2019,  0.4674,  ..., -0.1616,  0.3687, -0.2798],
         [ 0.0242,  0.0713,  0.5750,  ..., -0.2297,  0.3579, -0.2498],
         [ 0.5444, -0.1255,  0.3838,  ..., -0.2626,  0.0704, -0.6563]]],
       device='cuda:0'), pooler_output=tensor([[-0.0557

[W 2022-03-26 01:22:02,735] Trial 15 failed, because the value None could not be cast to float.
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2

Step,Training Loss


KeyboardInterrupt: 

In [13]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer.hyperparameter_search(
    direction="maximize"
    #compute_objective=my_objective
#     resources_per_trial={
#         "cpu": 1,
#         "gpu": 1
#     },
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    #search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=ASHAScheduler(metric="objective", mode="max"),
    #n_trials=8 # number of trials
)

[I 2022-03-26 09:55:28,648] A new study created in memory with name: no-name-252389af-e34b-4555-8972-8009b0177622
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "t

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8


[W 2022-03-26 09:55:40,009] Trial 0 failed, because the value None could not be cast to float.
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,

Step,Training Loss


...



Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8
[W 2022-03-26 09:56:25,704] Trial 10 failed, because the value None could not be cast to float.
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8
[W 2022-03-26 09:56:31,724] Trial 11 failed, because the value None could not be cast to float.
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads

Step,Training Loss


KeyboardInterrupt: 

In [16]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer.hyperparameter_search(
    direction="maximize",
    backend="ray"
    #compute_objective=my_objective
#     resources_per_trial={
#         "cpu": 1,
#         "gpu": 1
#     },
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    #search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=ASHAScheduler(metric="objective", mode="max"),
    #n_trials=8 # number of trials
)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.


TypeError: cannot pickle 'torch._C.Generator' object

In [14]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

trainer.hyperparameter_search(
    direction="maximize", 
    backend="optuna",
#     resources_per_trial={
#         "cpu": 1,
#         "gpu": 1
#     },
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    #search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=ASHAScheduler(metric="objective", mode="max"),
    #n_trials=8 # number of trials
)

[I 2022-03-25 17:12:20,441] A new study created in memory with name: no-name-83983eac-8c57-4a93-a391-1b39af9da1e2
Trial:
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\ng-ka/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "t

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 25
  Batch size = 8
[W 2022-03-25 17:12:26,037] Trial 0 failed because of the following error: ValueError('too many values to unpack (expected 2)')
Traceback (most recent call last):
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\optuna\study\_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\integrations.py", line 157, in _objective
    metrics = trainer.evaluate()
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\trainer.py", line 2257, in evaluate
    output = eval_loop(
  File "C:\Users\ng-ka\anaconda3\envs\cl-distilled\lib\site-packages\transformers\trainer.py", line 2431, in evaluation_loop
    loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_

input ids before
torch.Size([8, 2, 32])
tensor([[[  101, 16272,  2001,  2019, 11295,  3017,  1998,  2764,  1999,  1996,
           7364, 26866,  1037,  2177,  1997,  2714,  3241,  4177,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101, 16272,  2001,  2019, 11295,  3017,  1998,  2764,  1999,  1996,
           7364, 26866,  1037,  2177,  1997,  2714,  3241,  4177,  1012,   102,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0]],

        [[  101,  4821,  2023,  2150,  7262,  1998,  4675, 21572, 26638,  1012,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0],
         [  101,  4821,  2023,  2150,  7262,  1998,  4675, 21572, 26638,  1012,
            102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
   

ValueError: too many values to unpack (expected 2)

In [ ]:
model_path = 'trained_model/bert_cl'

#train_result = trainer.train(model_path=model_path)
train_result = trainer.train()

## 4. Evaluate Bert CL Model performance

In [ ]:
model.eval()

results = evaluate_model()
results

In [ ]:
    print(results["STS12"]["all"]["spearman"].keys())
    print('STS12 mean: ', results["STS12"]["all"]["spearman"]["mean"])
    print('STS12 wmean: ', results["STS12"]["all"]["spearman"]["wmean"])
    print('STS13 mean: ', results["STS13"]["all"]["spearman"]["mean"])
    print('STS13 wmean: ', results["STS13"]["all"]["spearman"]["wmean"])
    print('STS14 mean: ', results["STS14"]["all"]["spearman"]["mean"])
    print('STS14 wmean: ', results["STS14"]["all"]["spearman"]["wmean"])
    print('STS15 mean: ', results["STS15"]["all"]["spearman"]["mean"])
    print('STS15 wmean: ', results["STS15"]["all"]["spearman"]["wmean"])
    print('STSB: ', results["STSBenchmark"]["spearman"])